In [8]:
# imports, auth, and connection

import os
import json
import psycopg2
import pandas as pd
import uuid

conn_params = {
    'dbname': 'project_database',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost'
}

conn = psycopg2.connect(**conn_params)

In [14]:
# competitions

# filtering relevant comps
def filter_competitions(data):
    criteria = [
        {"competition_name": "La Liga", "season_name": "2020/2021"},
        {"competition_name": "La Liga", "season_name": "2019/2020"},
        {"competition_name": "La Liga", "season_name": "2018/2019"},
        {"competition_name": "Premier League", "season_name": "2003/2004"},
    ]
    filtered_data = [item for item in data if any(c["competition_name"] == item["competition_name"] and c["season_name"] == item["season_name"] for c in criteria)]
    return filtered_data

with open('open-data/data/competitions.json', 'r') as file:
    competitions_data = json.load(file)

competitions_to_insert = filter_competitions(competitions_data)

cur = conn.cursor()

insert_query = '''
    INSERT INTO Competitions (
        competition_id,
        season_id,
        country_name,
        competition_name,
        competition_gender,
        competition_youth,
        competition_international,
        season_name
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (competition_id, season_id) DO NOTHING;
'''

for item in competitions_to_insert:
    cur.execute(insert_query, (
        item["competition_id"],
        item["season_id"],
        item["country_name"],
        item["competition_name"],
        item["competition_gender"],
        item["competition_youth"],
        item["competition_international"],
        item["season_name"]
    ))

conn.commit()
cur.close()

In [6]:
# verifying competitions

query = """
SELECT * FROM Competitions;
"""

df_competitions = pd.read_sql_query(query, conn)
df_competitions


/var/folders/1b/q25_hkjn4ss017jjnfh0ytnw0000gn/T/ipykernel_94197/935086658.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_competitions = pd.read_sql_query(query, conn)


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name
0,11,90,Spain,La Liga,male,False,False,2020/2021
1,11,42,Spain,La Liga,male,False,False,2019/2020
2,11,4,Spain,La Liga,male,False,False,2018/2019
3,2,44,England,Premier League,male,False,False,2003/2004


In [39]:
# matches data

cur = conn.cursor()

seasons_dirs = {
    "11": ["90", "42", "4"],  # La Liga: Season IDs for 2020/2021, 2019/2020, 2018/2019
    "2": ["44"]  # Premier League: Season ID for 2003/2004
}

base_path = 'open-data/data/matches'

insert_sql = '''
INSERT INTO Matches (
    match_id, match_date, kick_off, country, competition, competition_id, season_id, 
    home_team, home_team_gender, home_team_manager_id, home_team_manager_name, home_team_group, home_team_country, 
    away_team, away_team_gender, away_team_manager_id, away_team_manager_name, away_team_group, away_team_country, 
    home_score, away_score, match_status, match_week, competition_stage, 
    stadium, stadium_country, referee_name, referee_country
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (match_id) DO NOTHING;
'''

def get_country_name(country_data):
    return country_data.get('name', 'na') if isinstance(country_data, dict) else 'na'

for comp_id, seasons in seasons_dirs.items():
    for season_id in seasons:
        file_path = os.path.join(base_path, comp_id, f"{season_id}.json")
        with open(file_path, 'r') as file:
            matches_data = json.load(file)
            for match in matches_data:
                try:
                    parameters = (
                        match.get("match_id"),
                        match.get("match_date"),
                        match.get("kick_off"),
                        match["competition"].get("country_name"),
                        match["competition"].get("competition_name"),
                        match["competition"].get("competition_id"),
                        match["season"].get("season_id"),
                        match["home_team"].get("home_team_name"),
                        match["home_team"].get("home_team_gender"),
                        match["home_team"].get("managers", [{}])[0].get("id", None),
                        match["home_team"].get("managers", [{}])[0].get("name", None),
                        match["home_team"].get("home_team_group"),
                        get_country_name(match["home_team"].get("country")),
                        match["away_team"].get("away_team_name"),
                        match["away_team"].get("away_team_gender"),
                        match["away_team"].get("managers", [{}])[0].get("id", None),
                        match["away_team"].get("managers", [{}])[0].get("name", None),
                        match["away_team"].get("away_team_group"),
                        get_country_name(match["away_team"].get("country")),
                        match.get("home_score"),
                        match.get("away_score"),
                        match.get("match_status"),
                        match.get("match_week"),
                        match["competition_stage"].get("name"),
                        match.get("stadium", {}).get("name"),
                        match.get("stadium", {}).get("country", {}).get("name"),
                        match.get("referee", {}).get("name", None),
                        match.get("referee", {}).get("country", {}).get("name", None)
                    )

                    cur.execute(insert_sql, parameters)
                except psycopg2.Error as e:
                    print(f"An error occurred: {e}")
                    conn.rollback()
                    continue 
                
            conn.commit()

conn.commit()
cur.close()

In [40]:
# matches verification

query = """
SELECT * FROM Matches;
"""

df_matches = pd.read_sql_query(query, conn)
df_matches

/var/folders/1b/q25_hkjn4ss017jjnfh0ytnw0000gn/T/ipykernel_2318/998478867.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_matches = pd.read_sql_query(query, conn)


,match_id,match_date,kick_off,country,competition,competition_id,season_id,home_team,home_team_gender,home_team_manager_id,...,away_team_country,home_score,away_score,match_status,match_week,competition_stage,stadium,stadium_country,referee_name,referee_country
0,3749310,2003-12-29,21:00:00,England,Premier League,2,44,Southampton,male,3722.0,...,England,0,1,available,1,Regular Season,St. Mary''s Stadium,England,None,None
1,3749493,2003-08-16,16:00:00,England,Premier League,2,44,Arsenal,male,577.0,...,England,2,1,available,1,Regular Season,Highbury Stadium,England,None,None
2,3749434,2003-11-22,16:00:00,England,Premier League,2,44,Birmingham City,male,37.0,...,England,0,3,available,1,Regular Season,St. Andrews Stadium,England,None,None
3,3749192,2003-11-30,15:00:00,England,Premier League,2,44,Arsenal,male,577.0,...,England,0,0,available,14,Regular Season,Highbury Stadium,England,None,None
4,3749196,2003-11-08,16:00:00,England,Premier League,2,44,Arsenal,male,577.0,...,England,2,1,available,12,Regular Season,Highbury Stadium,England,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,3749552,2003-09-21,17:05:00,England,Premier League,2,44,Manchester United,male,790.0,...,England,0,0,available,1,Regular Season,Old Trafford,England,None,None
136,3749296,2003-09-13,16:00:00,England,Premier League,2,44,Arsenal,male,577.0,...,England,1,1,available,5,Regular Season,Highbury Stadium,England,None,None
137,3749454,2003-08-31,17:05:00,England,Premier League,2,44,Manchester City,male,4202.0,...,England,1,2,available,1,Regular Season,Etihad Stadium,England,None,None
138,3749276,2004-02-21,13:30:00,England,Premier League,2,44,Chelsea,male,60.0,...,England,1,2,available,2,Regular Season,Stamford Bridge,England,None,None


In [42]:
# get relevant matches as only events/lineups from these games will matter
cur = conn.cursor()

# SQL query to select match_id from matches
sql_query = "SELECT match_id FROM matches;"
cur.execute(sql_query)

# Fetch all the results
results = cur.fetchall()

match_ids = set()

# iterate over the results and add each match_id to the set
for result in results:
    match_ids.add(result[0])

print(match_ids)
print(len(match_ids))

cur.close()

{3773457, 303634, 3773466, 3749403, 3773474, 303652, 3773477, 303664, 303666, 3749431, 3773497, 303674, 3749434, 303680, 303682, 3749448, 15946, 3749453, 3749454, 303696, 3773523, 303700, 15956, 3773526, 3749462, 3749465, 303707, 303715, 15973, 15978, 3773547, 303725, 3773552, 15986, 303731, 3749493, 3773565, 15998, 3773571, 16010, 3773585, 3773586, 3773587, 3749522, 3749526, 16023, 3749528, 3773593, 3773597, 16029, 3749552, 16056, 3773625, 3749052, 3773631, 16073, 3749068, 16079, 3749590, 16086, 3773656, 3764440, 3749079, 3773660, 3773661, 16095, 3773665, 3749603, 3773672, 16109, 3749108, 16120, 3773689, 3749117, 3773695, 3749631, 16131, 16136, 3749642, 3749133, 303377, 16149, 16157, 3749153, 303400, 16173, 16182, 303421, 16190, 16196, 303430, 3749192, 3749196, 16205, 16215, 303451, 16231, 303470, 16240, 303473, 3749233, 303479, 16248, 3749246, 303487, 303493, 3749253, 16265, 3749257, 303504, 16275, 3749274, 303516, 303517, 3749278, 3749276, 16289, 303524, 303532, 3749296, 16306, 3764

In [44]:
# handling relevant files only to keep size of repo manageable

base_path = 'open-data/data'
events_path = os.path.join(base_path, 'events')
lineups_path = os.path.join(base_path, 'lineups')

def clean_directory(directory_path):
    for filename in os.listdir(directory_path):
        match_id = filename.split('.')[0]
        
        if match_id.isdigit():
            match_id = int(match_id)
            
            if match_id not in match_ids:
                file_path = os.path.join(directory_path, filename)
                print(f"Deleting: {file_path}")
                os.remove(file_path)
        else:
            print(f"Skipping non-match file: {filename}")

# clean events directory
clean_directory(events_path)

# clean lineups directory
clean_directory(lineups_path)

print("Cleanup completed.")

Deleting: open-data/data/events/3879600.json
Deleting: open-data/data/events/3901250.json
Deleting: open-data/data/events/3754300.json
Deleting: open-data/data/events/2275050.json
Deleting: open-data/data/events/19795.json
Deleting: open-data/data/events/3825894.json
Deleting: open-data/data/events/7298.json
Deleting: open-data/data/events/3753996.json
Deleting: open-data/data/events/3900504.json
Deleting: open-data/data/events/3753979.json
Deleting: open-data/data/events/265958.json
Deleting: open-data/data/events/3890360.json
Deleting: open-data/data/events/3879745.json
Deleting: open-data/data/events/3754245.json
Deleting: open-data/data/events/69182.json
Deleting: open-data/data/events/3902240.json
Deleting: open-data/data/events/3890419.json
Deleting: open-data/data/events/18242.json
Deleting: open-data/data/events/69301.json
Deleting: open-data/data/events/3901196.json
Deleting: open-data/data/events/3775631.json
Deleting: open-data/data/events/3825601.json
Deleting: open-data/da

In [45]:
# verifying that the new file count matches with the length of match_ids

base_path = 'open-data/data'
events_path = os.path.join(base_path, 'events')
lineups_path = os.path.join(base_path, 'lineups')

def count_files(directory_path):
    return len([name for name in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, name))])

events_count = count_files(events_path)
lineups_count = count_files(lineups_path)

# print the counts
print(f"Number of files in 'events': {events_count}")
print(f"Number of files in 'lineups': {lineups_count}")

Number of files in 'events': 140
Number of files in 'lineups': 140


In [2]:
# events unique properties and their value depths within event payloads

events_dict = {
    "Event": {
        "event_id": "id",
        "match_id": "file_name",
        "event_index":"index",
        "game_period":"period",
        "start_time": "timestamp",
        "min": "minute",
        "sec": "second",
        "possession": "possession",
        "possession_team_name": ("possession_team", "name"),
        "play_pattern_name": ("play_pattern", "name"),
        "team_name": ("team", "name"),
        "player_name": ("player", "name"),
        "player_position": ("position", "name"),
        "location_x": ("location", 0),
        "location_y": ("location", 1),
        "under_pressure": "under_pressure",
        "out_of_bounds": "out",
        "duration": "duration",
    },
    "Pass": {
        "recipient_name": ("recipient", "name"),
        "pass_length": "length",
        "pass_angle": "angle",
        "pass_height": ("height", "name"),
        "end_location_x": ("end_location", 0),
        "end_location_y": ("end_location", 1),
        "deflected": "deflected",
        "is_backheel": "backheel",
        "is_cross": "cross",
        "is_cutback": "cut-back",
        "is_switch": "switch",
        "shot_assist": "shot-assist",
        "goal_assist": "goal-assist",
        "pass_body_part": ("body_part", "name"),
        "pass_type": ("type", "name"),
        "outcome_name": ("outcome", "name"),
        "technique": ("technique", "name"),
    },
    "Shot": {
        "end_location_x": ("end_location", 0),
        "end_location_y": ("end_location", 1),
        "end_location_z": ("end_location", 2),
        "aerial_won": "aerial_won",
        "follows_dribble": "follows_dribble",
        "first_time": "first_time",
        "open_goal": "open_goal",
        "statsbomb_xg": "statsbomb_xg",
        "deflected": "deflected",
        "technique_name": ("technique", "name"),
        "body_part_name": ("body_part", "name"),
        "type_name": ("type", "name"),
        "outcome_name": ("outcome", "name"),
    },
    "Dribble": {
        "overrun": "overrun",
        "nutmeg": "nutmeg",
        "no_touch": "no_touch",
        "outcome": ("outcome", "name"),
    },
    "Dribbled_Past": {
        "counterpress": "counterpress",
    },
    "Bad_Behaviour": {
        "card_colour": ("card", "name"),
    },
    "Ball_Receipt": {
        "outcome": ("outcome", "name"),
    },
    "Ball_Recovery": {
        "offensive": "offensive",
        "recovery_failure": "recovery_failure",
    },
    "Block": {
        "deflection": "deflection",
        "offensive": "offensive",
        "save_block": "save_block",
        "counterpress": "counterpress",
    },
    "Carry": {
        "end_location_x": ("end_location", 0),
        "end_location_y": ("end_location", 1),
    },
    "Clearance": {
        "aerial_won": "aerial_won",
        "body_part_name": ("body_part", "name"),
    },
    "Duel": {
        "counterpress": "counterpress",
        "duel_type": ("type", "name"),
        "outcome": ("outcome", "name"),
    },
    "Foul_Committed": {
        "counterpress": "counterpress",
        "offensive": "offensive",
        "foul_type": ("type", "name"),
        "advantage": "advantage",
        "penalty": "penalty",
        "card_colour": ("card", "name"),
    },
    "Foul_Won": {
        "defensive": "defensive",
        "advantage": "advantage",
        "penalty": "penalty",
    },
    "Goalkeeper": {
        "position": ("position", "name"),
        "technique": ("technique", "name"),
        "body_part_name": ("body_part", "name"),
        "goalkeeper_event_type": ("type", "name"),
        "outcome": ("outcome", "name"),
    },
    "Half_End": {
        "early_video_end": "early_video_end",
        "match_suspended": "match_suspended",
    },
    "Half_Start": {
        "late_video_start": "late_video_start",
    },
    "Injury_Stoppage": {
        "in_chain": "in_chain",
    },
    "Interception": {
        "outcome": ("outcome", "name"),
    },
    "Miscontrol": {
        "aerial_won": "aerial_won",
    },
    "Player_Off": {
        "permanent": "permanent",
    },
    "Pressure": {
        "counterpress": "counterpress",
    },
    "Substitution": {
        "replacement": ("replacement", "name"),
        "outcome": ("outcome", "name"),
    }
}

In [3]:
# inserting events data. file took me ~2 minutes to execute

conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

def get_value_from_json(event_json, keys):
    value = event_json
    for key in keys:
        try:
            if isinstance(value, dict):
                value = value[key]
            elif isinstance(key, int):  # Handling list indices
                value = value[key]
        except (KeyError, TypeError, IndexError):
            return None
    return value

def insert_event_data(event_json, match_id, event_type, cur, log_file):
    try:
        columns = ["match_id"]
        values = [match_id]

        # log_file.write(f"\nProcessing event: {event_json.get('id')}\n")
        # log_file.write(f"Event Type: {event_type}\n")

        # Add common event properties
        event_properties = events_dict.get("Event", {})
        for col, path in event_properties.items():
            value = get_value_from_json(event_json, path if isinstance(path, tuple) else [path])
            # log_file.write(f"Common Property: {col}, Extracted Value: {value}\n")
            if value is not None:
                columns.append(col)
                values.append(value)

        # Add event-specific properties
        event_specific_properties = events_dict.get(event_type, {})
        event_type_json = event_json.get(event_type.lower(), {})

        for col, path in event_specific_properties.items():
            if isinstance(path, tuple):
                # nested attributes are fetched from within the event-specific JSON
                value = get_value_from_json(event_type_json, path)
            else:
                json_key = path if isinstance(path, str) else path[1]
                value = event_type_json.get(json_key)

            # log_file.write(f"Event-specific Property: {col}, Extracted Value: {value}\n")
            if value is not None:
                columns.append(col)
                values.append(value)

        query = f"INSERT INTO {event_type} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(values))})"
        cur.execute(query, values)
        conn.commit()
        # log_file.write(f"Successful Insert: {query} Values: {values}\n")

    except Exception as e:
        conn.rollback()
        # log_file.write(f"Error inserting event {event_json.get('id', 'Unknown')} for match {match_id}: {e}\n")

def process_events_folder(folder_path, cur, log_file_path):
    with open(log_file_path, 'a') as log_file:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.json'):
                    match_id = int(file.split('.')[0])
                    with open(os.path.join(root, file), 'r') as f:
                        events = json.load(f)
                        for event_json in events:
                            event_type_name = get_value_from_json(event_json, ["type", "name"])
                            event_type_name = event_type_name.replace(' ', '_').replace('*', '')
                            if event_type_name == "Goal_Keeper":
                                event_type_name = "Goalkeeper"
                            if event_type_name in events_dict:
                                insert_event_data(event_json, match_id, event_type_name, cur, log_file)
                            # else:
                                # log_file.write(f"Unprocessed event type: {event_type_name}\n")
                                
folder_path = 'open-data/data/events'
log_file_path = 'logfile.txt'
process_events_folder(folder_path, cur, log_file_path)

conn.close()

In [5]:
# lineups data
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

def insert_lineup_data(file_path, match_id, cur):
    try:
        with open(file_path, 'r') as file:
            lineup_details = json.load(file)
        
        query = "INSERT INTO Lineups (match_id, lineup_details) VALUES (%s, %s)"
        values = (match_id, json.dumps(lineup_details))
        
        cur.execute(query, values)
        conn.commit()
        
    except Exception as e:
        conn.rollback()
        print(f"Error inserting lineup for match {match_id}: {e}")

def process_lineups_folder(folder_path, cur):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.json'):
                match_id = int(file.split('.')[0])
                file_path = os.path.join(root, file)
                insert_lineup_data(file_path, match_id, cur)

folder_path = 'open-data/data/lineups'
process_lineups_folder(folder_path, cur)


conn.close()

In [7]:
# with the above script, all data should be loaded to the db
# dump command:
# pg_dump --file "/Users/saipaladugu/Desktop/dbexport.sql" --host "localhost" --port "5432" --username "postgres" --verbose --format=p "project_database"
cur.close()